In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import os

# CSV 파일 읽기
df = pd.read_csv("dc_(ID,Artist).csv")

# 결과를 저장할 리스트
results = []

# 결과 파일 경로
output_file = "output.csv"

# 결과 파일이 존재할 경우 기존 데이터를 읽어옴
if os.path.exists(output_file):
    results_df = pd.read_csv(output_file)
else:
    results_df = pd.DataFrame(columns=["아티스트", "dcinside_ID", "작성일", "조회수", "추천수"])

# 첫 번째 아티스트에 대해 데이터 수집
for index, row in df.iterrows():
    artist_name = row["Artist"]
    artist_id = row["dcinside_ID"]
    print(f'수집 중: 아티스트 "{artist_name}" (ID: {artist_id})')

    page = 1

    scrapying = True

    while scrapying:
        url = f"https://gall.dcinside.com/board/lists/?id={artist_id}&page={page}"
        print(url)

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }

        max_retries = 3
        for attempt in range(max_retries):
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                break
            print(f"요청 실패, 재시도 {attempt + 1}/{max_retries} - 상태 코드: {response.status_code}")
            time.sleep(2)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            posts = soup.find_all("tr", class_="ub-content us-post")

            if not posts:
                print(f"    더 이상 게시글이 없습니다. 아티스트: {artist_name}, 페이지: {page}. 수집 종료.")
                break
            
            for post in posts:
                if post.get("data-type") == "icon_notice":
                    print("    공지 게시글 발견: 건너뜁니다.")
                    continue

                post_date = post.find("td", class_="gall_date").text.strip()

                if len(post_date) == 5:
                    if post_date[2] == '.':
                        formatted_post_date = f"24.{post_date}"
                    else:
                        print("    작성일 형식 오류: 건너뜁니다.")
                        continue
                else:
                    formatted_post_date = post_date

                cutoff_date = "23.09.23"
                formatted_cutoff_date = "23.09.23"

                post_date_dt = datetime.strptime(formatted_post_date, "%y.%m.%d")
                cutoff_date_dt = datetime.strptime(formatted_cutoff_date, "%y.%m.%d")

                if post_date_dt <= cutoff_date_dt:
                    print(f"    종료 조건 충족: 작성일 {formatted_post_date}. 수집 완료.")
                    scrapying = False
                    break
                
                views = post.find("td", class_="gall_count").text.strip()
                recommendations = post.find("td", class_="gall_recommend").text.strip()
                
                results.append({
                    "아티스트": artist_name,
                    "dcinside_ID": artist_id,
                    "작성일": formatted_post_date,
                    "조회수": views,
                    "추천수": recommendations
                })
                print(f"    수집됨: 작성일 {formatted_post_date}, 조회수 {views}, 추천수 {recommendations}")

            page += 1
            print(f"    다음 페이지: {page}")
            time.sleep(2)
        else:
            print(f"{artist_name}의 데이터 가져오기 오류: {response.status_code}")
            break

# 수집된 데이터를 DataFrame으로 변환
results_df = pd.concat([results_df, pd.DataFrame(results)], ignore_index=True)

# 결과를 CSV 파일로 저장 (기존 파일에 추가)
results_df.to_csv(output_file, index=False)
print("데이터 수집 완료. 결과를 'output.csv'로 저장했습니다.")
